### Load the network for transer learning

In [8]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Model

def create_model():
    local_weights_file = './inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

# Initialize the base model.
# Set the input shape and remove the dense layers.
    pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

# Load the pre-trained weights you downloaded.
    pre_trained_model.load_weights(local_weights_file)

# Freeze the weights of the layers.
    for layer in pre_trained_model.layers:
      layer.trainable = False
    
    #choose mixed7 as the final convo layer
    last_layer=pre_trained_model.get_layer('mixed7')
    last_output=last_layer.output
    
    #create the model that we will use by adding some dense layers
    from tensorflow.keras.optimizers import RMSprop
    from tensorflow.keras import Model

    # Flatten the output layer to 1 dimension
    x=layers.Flatten()(last_output)

    # Add a fully connected layer with 1,024 hidden units and ReLU activation
    x=layers.Dense(1024,activation='relu')(x)

    # Add a dropout rate of 0.2
    x = layers.Dropout(0.2)(x)      

    # Add a final sigmoid layer for classification
    x = layers.Dense  (1, activation='sigmoid')(x)           

    # Append the dense network to the base model
    model = Model(pre_trained_model.input, x) 
    
    return model

    
    

In [1]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers

# Set the weights file you downloaded into a variable
local_weights_file = './inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

# Initialize the base model.
# Set the input shape and remove the dense layers.
pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

# Load the pre-trained weights you downloaded.
pre_trained_model.load_weights(local_weights_file)

# Freeze the weights of the layers.
for layer in pre_trained_model.layers:
  layer.trainable = False

In [2]:
#choose mixed7 as the final convo layer
last_layer=pre_trained_model.get_layer('mixed7')
last_output=last_layer.output

In [3]:
#create the model that we will use by adding some dense layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Model

# Flatten the output layer to 1 dimension
x=layers.Flatten()(last_output)

# Add a fully connected layer with 1,024 hidden units and ReLU activation
x=layers.Dense(1024,activation='relu')(x)

# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)      

# Add a final sigmoid layer for classification
x = layers.Dense  (1, activation='sigmoid')(x)           

# Append the dense network to the base model
model = Model(pre_trained_model.input, x) 


In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

### Prepare the data

In [4]:
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Define our example directories and files
base_dir = './cats_and_dogs_filtered'

train_dir = os.path.join( base_dir, 'train')
validation_dir = os.path.join( base_dir, 'validation')

# Directory with training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats') 

# Directory with training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs') 

# Directory with validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats') 

# Directory with validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


### Set the callback func so that weights of the model will be saved every 2 epochs

In [5]:

model.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])
    

In [6]:
import tensorflow as tf
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 20

# Create a callback that saves the model's weights every 2 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=2*batch_size)



# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback

history = model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 100,
            epochs = 20,
            validation_steps = 50,
            callbacks=[cp_callback],
            verbose = 1)

Epoch 1/20
 40/100 [===========>..................] - ETA: 12s - loss: 0.4752 - accuracy: 0.8325
Epoch 00001: saving model to training_2\cp-0001.ckpt
 80/100 [=======================>......] - ETA: 5s - loss: 0.3935 - accuracy: 0.8506
Epoch 00001: saving model to training_2\cp-0001.ckpt
100/100 [==============================] - 39s 310ms/step - loss: 0.3669 - accuracy: 0.8590 - val_loss: 0.0867 - val_accuracy: 0.9680
Epoch 2/20
 20/100 [=====>........................] - ETA: 12s - loss: 0.2135 - accuracy: 0.9100
Epoch 00002: saving model to training_2\cp-0002.ckpt
 60/100 [=================>............] - ETA: 7s - loss: 0.2370 - accuracy: 0.9050
Epoch 00002: saving model to training_2\cp-0002.ckpt
100/100 [==============================] - ETA: 0s - loss: 0.2189 - accuracy: 0.9125
Epoch 00002: saving model to training_2\cp-0002.ckpt
100/100 [==============================] - 25s 249ms/step - loss: 0.2189 - accuracy: 0.9125 - val_loss: 0.0949 - val_accuracy: 0.9690
Epoch 3/20
 40/100

In [7]:
os.listdir(checkpoint_dir)

['checkpoint',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0001.ckpt.data-00000-of-00001',
 'cp-0001.ckpt.index',
 'cp-0002.ckpt.data-00000-of-00001',
 'cp-0002.ckpt.index',
 'cp-0003.ckpt.data-00000-of-00001',
 'cp-0003.ckpt.index',
 'cp-0004.ckpt.data-00000-of-00001',
 'cp-0004.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.index',
 'cp-0006.ckpt.data-00000-of-00001',
 'cp-0006.ckpt.index',
 'cp-0007.ckpt.data-00000-of-00001',
 'cp-0007.ckpt.index',
 'cp-0008.ckpt.data-00000-of-00001',
 'cp-0008.ckpt.index',
 'cp-0009.ckpt.data-00000-of-00001',
 'cp-0009.ckpt.index',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.index',
 'cp-0011.ckpt.data-00000-of-00001',
 'cp-0011.ckpt.index',
 'cp-0012.ckpt.data-00000-of-00001',
 'cp-0012.ckpt.index',
 'cp-0013.ckpt.data-00000-of-00001',
 'cp-0013.ckpt.index',
 'cp-0014.ckpt.data-00000-of-00001',
 'cp-0014.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.index',
 'cp-0016.ckpt.data-00000

### Check the accuracy of model with handpicked image

In [9]:
# create a model with weights saved in epoch 13
model_epoch_13=create_model()

In [10]:
model_epoch_13.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

In [11]:
checkpoint_path_epoch_13 = "training_2/cp-0013.ckpt"
# Loads the weights
model_epoch_13.load_weights(checkpoint_path_epoch_13)


In [12]:
# Re-evaluate the model
loss, acc = model.evaluate(validation_generator, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

50/50 - 2s - loss: 0.1006 - accuracy: 0.9710
Restored model, accuracy: 97.10%


In [18]:
from ipywidgets import FileUpload
upload=FileUpload(multiple=True)
display(upload)

FileUpload(value={}, description='Upload', multiple=True)

In [27]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
for fn in upload.value.keys():
    path=os.path.join('./test/',fn)
    img=image.load_img(path,target_size=(150,150))
    x=image.img_to_array(img)
    x/=255
    x=np.expand_dims(x,axis=0)
    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    print(classes[0])
    
    if classes[0]>0.5:
        print(fn + " is a dog")
        
    else:
        print(fn + " is a cat")
    

[0.00065097]
cat sleeping on couch 4.jpg is a cat
[9.839734e-05]
cat sleeping on couch 5.jpg is a cat
[0.9997191]
cat sleeping on couch.jpg is a dog


In [28]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
for fn in upload.value.keys():
    path=os.path.join('./test/',fn)
    img=image.load_img(path,target_size=(150,150))
    x=image.img_to_array(img)
    x/=255
    x=np.expand_dims(x,axis=0)
    images = np.vstack([x])
    classes = model_epoch_13.predict(images, batch_size=10)
    print(classes[0])
    
    if classes[0]>0.5:
        print(fn + " is a dog")
        
    else:
        print(fn + " is a cat")

[0.7723841]
cat sleeping on couch 4.jpg is a dog
[5.879124e-06]
cat sleeping on couch 5.jpg is a cat
[0.99999475]
cat sleeping on couch.jpg is a dog
